In [1]:
# -*- coding: utf-8 -*-
"""Alexnet_code.ipynb

Automatically generated by Colaboratory.

Original file is located at
    https://colab.research.google.com/drive/1ST_yrafE2p_4JHcbvSpBBYMW1lWIw5hD
"""

from __future__ import print_function
import keras
import pandas as pd
from keras.layers import Dense, Conv2D, BatchNormalization, Activation
from keras.layers import AveragePooling2D, Input, Flatten
from keras.optimizers import Adam
from keras.callbacks import ModelCheckpoint, LearningRateScheduler
from keras.callbacks import ReduceLROnPlateau
from keras.callbacks import EarlyStopping
from keras.callbacks import ModelCheckpoint
from keras.preprocessing.image import ImageDataGenerator
from keras.regularizers import l2
from keras import backend as K
from keras.models import Model
from keras.models import load_model
from keras.utils import to_categorical
import numpy as np
import os
import time
from openpyxl import load_workbook
from keras.models import Sequential
from keras.layers import Dense, Activation, Dropout, Flatten,Conv2D, MaxPooling2D
from keras.layers import BatchNormalization
from PIL import Image
import tensorflow as tf

import xlsxwriter
import cv2
from random import shuffle




start1= time.time()

# Training parameters
batch_size = 10  # orig paper trained all networks with batch_size=128
epochs = 50
data_augmentation = False
num_classes = 12
test_img_num =2083
result_excel_link = '/Users/sayantansikdar/Developer/FRUITS_MASTER/Alexnet_Result/fruit_batch10_epoch50.xlsx'
Train_DIR= '/Users/sayantansikdar/Developer/FRUITS_MASTER/FINAL DATA_Fruit/TRAIN'
TEST_DIR= '/Users/sayantansikdar/Developer/FRUITS_MASTER/FINAL DATA_Fruit/TEST'
label_link= '/Users/sayantansikdar/Developer/FRUITS_MASTER/FINAL_encoded_file.xlsx'

# Subtracting pixel mean improves accuracy
subtract_pixel_mean = True

# Model parameter
# ----------------------------------------------------------------------------
#           |      | 200-epoch | Orig Paper| 200-epoch | Orig Paper| sec/epoch
# Model     |  n   | ResNet v1 | ResNet v1 | ResNet v2 | ResNet v2 | GTX1080Ti
#           |v1(v2)| %Accuracy | %Accuracy | %Accuracy | %Accuracy | v1 (v2)
# ----------------------------------------------------------------------------
# ResNet20  | 3 (2)| 92.16     | 91.25     | -----     | -----     | 35 (---)
# ResNet32  | 5(NA)| 92.46     | 92.49     | NA        | NA        | 50 ( NA)
# ResNet44  | 7(NA)| 92.50     | 92.83     | NA        | NA        | 70 ( NA)
# ResNet56  | 9 (6)| 92.71     | 93.03     | 93.01     | NA        | 90 (100)
# ResNet110 |18(12)| 92.65     | 93.39+-.16| 93.15     | 93.63     | 165(180)
# ResNet164 |27(18)| -----     | 94.07     | -----     | 94.54     | ---(---)
# ResNet1001| (111)| -----     | 92.39     | -----     | 95.08+-.14| ---(---)
# ---------------------------------------------------------------------------


# Model version
# Orig paper: version = 1 (ResNet v1), Improved ResNet: version = 2 (ResNet v2)
version = 2
n = 2
# Computed depth from supplied model parameter n
if version == 1:
    depth = n * 6 + 2
elif version == 2:
    depth = n * 9 + 2

# Model name, depth and version
model_type = 'AlexNet%dv%d' % (depth, version)

# Load the CIFAR10 data.
##(x_train, y_train), (x_test, y_test) = cifar10.load_data()



def label_img(name):
    # print(name)
    filename1 = os.fsdecode(name)
    # print(filename1)
    s=[]
    s= filename1.split('_')
    # print(s)
    # print(int(s[0]))
    #word_label = name.split('-')[0]
    #if word_label == 'golden_retriever': return np.array([1, 0])
    #elif word_label == 'shetland_sheepdog' : return np.array([0, 1])
    wb = load_workbook(label_link)
    ws = wb.active
    #for cell in ws.columns[1]:  #here column 9 is value is what i am testing which is Column X of my example.
    for x in range(2, 18454):
        #for y in range(1,):
        cell= ws.cell(row=x, column=1)
        # print(cell.value)
        # print(cell.value)
        if cell.value == int(s[0]) :
               #print ws.cell(column=12).value
                #a= ws.columns(row).value
                #b= ws.columns(col).value
                cell2= ws.cell(row=x, column=2)

                label1= cell2.value
                # print(label1)
                break
        #else:
                #print('hi')
    return label1

def load_training_data():
    train_data = []
    for img in os.listdir(Train_DIR):
        #print(img)
        temp2 = img.replace('.png','')
        label = label_img(temp2)
        #print(label)
        path = os.path.join(Train_DIR, img)
        #if "DS_Store" not in path:
        #img = Image.open(path)
        # print(path)
        # print(label)
        img2= cv2.imread(path)
        # cv2_imshow(img2)
        #img = img.convert('L')
        # img = img.resize((IMG_SIZE, IMG_SIZE, 3), Image.ANTIALIAS)
        #path3= 'D:/research phd/agriculture research/2nd phase of research/leaf phenotyping/training_441_segmented_polar_A1_A2_A4'
        resized_img1 = cv2.resize(img2, (224,224), interpolation = cv2.INTER_AREA)
        #cv2.imwrite(path3+ '/' + temp2 , resized_img1 )
        train_data.append([np.array(resized_img1), label])

    shuffle(train_data)
    return train_data


train_data = load_training_data()

# s = [i[1] for i in train_data]
# print(train_data)

# df2 = pd.DataFrame(np.array(df2),columns=['Label'])
# df2['Label'] = pd.Categorical(df2.Label)
IMG_SIZE = 224
# trainImages = np.array([i[0] for i in train_data],dtype=object) #.reshape(-1, IMG_SIZE, IMG_SIZE, 1)
trainImages = np.array([i[0] for i in train_data])
# df = pd.read_csv()
# trainLabels = tf.convert_to_tensor(np.array([i[1] for i in train_data]).astype(np.int_))
trainLabels = np.array([i[1] for i in train_data])
print(trainLabels[2]) #successful till here
#print(trainLabels)
# trainLabels = to_categorical(trainLabels)                                                          #problem here
trainLabels = keras.utils.to_categorical(trainLabels, num_classes)
print(trainLabels[2])



def load_test_data():
    test_data = []
    for img in os.listdir(TEST_DIR):
        temp1 = img.replace('.png','')
        # print(temp1)
        label = label_img(temp1)
        path = os.path.join(TEST_DIR, img)
        # temp1 = img
        #if "DS_Store" not in path:
        #img = Image.open(path)
        img1= cv2.imread(path)
        #img = img.convert('L')
        #img = img.resize((IMG_SIZE, IMG_SIZE, 3), Image.ANTIALIAS)
        resized_img2 = cv2.resize(img1, (224,224), interpolation = cv2.INTER_AREA)
        #path2= 'D:/research phd/agriculture research/2nd phase of research/leaf phenotyping/testing folder_segmented_resized'
        #cv2.imwrite(path2+ '/' + temp1 , resized_img2 )
        test_data.append([np.array(resized_img2), label,temp1])
        # print(label)

    shuffle(test_data)
    return test_data


test_data = load_test_data()
#plt.imshow(test_data[10][0], cmap = 'gist_gray')


# # convert it to tensorflow              ########################
# tensor1 = tf.convert_to_tensor(numpy_array)
# print(tensor1)

# testImages = np.array([i[0] for i in test_data],dtype=object) #.reshape(-1, IMG_SIZE, IMG_SIZE, 1)
# testLabels = tf.convert_to_tensor(np.array([i[1] for i in test_data]).astype(np.int_))

testImages = np.array([i[0] for i in test_data])
testLabels = np.array([i[1] for i in test_data])
test_img_name = np.array([i[2] for i in test_data])


#testLabels = keras.utils.to_categorical(testLabels, num_classes)

# Input image dimensions.
input_shape = trainImages.shape[1:]

# Normalize data.
##trainImages = trainImages.astype('float32') / 255
##testImages = testImages.astype('float32') / 255

# If subtract pixel mean is enabled
##if subtract_pixel_mean:
    ##trainImages_mean = np.mean(trainImages, axis=0)
    ##trainImages -= trainImages_mean
    ##testImages -= trainImages_mean

#print('x_train shape:', x_train.shape)
#print(x_train.shape[0], 'train samples')
#print(x_test.shape[0], 'test samples')
#print('y_train shape:', y_train.shape)

# Convert class vectors to binary class matrices.
#y_train = keras.utils.to_categorical(y_train, num_classes)
#y_test = keras.utils.to_categorical(y_test, num_classes)

def lr_schedule(epoch):
    """Learning Rate Schedule

    Learning rate is scheduled to be reduced after 80, 120, 160, 180 epochs.
    Called automatically every epoch as part of callbacks during training.

    # Arguments
        epoch (int): The number of epochs

    # Returns
        lr (float32): learning rate
    """
    lr = 1e-3
    if epoch > 180:
        lr *= 0.5e-3
    elif epoch > 160:
        lr *= 1e-3
    elif epoch > 120:
        lr *= 1e-2
    elif epoch > 80:
        lr *= 1e-1
    print('Learning rate: ', lr)
    return lr


model = Sequential()

# 1st Convolutional Layer
model.add(Conv2D(filters=96, input_shape=(224,224,3), kernel_size=(11,11),strides=(4,4), padding='valid'))
model.add(Activation('relu'))
# Pooling
model.add(MaxPooling2D(pool_size=(2,2), strides=(2,2), padding='valid'))
# Batch Normalisation before passing it to the next layer
model.add(BatchNormalization())

# 2nd Convolutional Layer
model.add(Conv2D(filters=256, kernel_size=(11,11), strides=(1,1), padding='valid'))
model.add(Activation('relu'))
# Pooling
model.add(MaxPooling2D(pool_size=(2,2), strides=(2,2), padding='valid'))
# Batch Normalisation
model.add(BatchNormalization())

# 3rd Convolutional Layer
model.add(Conv2D(filters=384, kernel_size=(3,3), strides=(1,1), padding='valid'))
model.add(Activation('relu'))
# Batch Normalisation
model.add(BatchNormalization())

# 4th Convolutional Layer
model.add(Conv2D(filters=384, kernel_size=(3,3), strides=(1,1), padding='valid'))
model.add(Activation('relu'))
# Batch Normalisation
model.add(BatchNormalization())

# 5th Convolutional Layer
model.add(Conv2D(filters=256, kernel_size=(3,3), strides=(1,1), padding='valid'))
model.add(Activation('relu'))
# Pooling
model.add(MaxPooling2D(pool_size=(2,2), strides=(2,2), padding='valid'))
# Batch Normalisation
model.add(BatchNormalization())

# Passing it to a dense layer
model.add(Flatten())
# 1st Dense Layer
model.add(Dense(4096, input_shape=(224*224*3,)))
model.add(Activation('relu'))
# Add Dropout to prevent overfitting
model.add(Dropout(0.4))
# Batch Normalisation
model.add(BatchNormalization())
# 2nd Dense Layer
model.add(Dense(4096))
model.add(Activation('relu'))
# Add Dropout
model.add(Dropout(0.4))
# Batch Normalisation
model.add(BatchNormalization())
# 3rd Dense Layer
model.add(Dense(1000))
model.add(Activation('relu'))
# Add Dropout
model.add(Dropout(0.4))
# Batch Normalisation
model.add(BatchNormalization())
# 4th Dense Layer
model.add(Dense(100))
model.add(Activation('relu'))
# Add Dropout
model.add(Dropout(0.4))
# Batch Normalisation
model.add(BatchNormalization())
# Output Layer
model.add(Dense(num_classes))
model.add(Activation('softmax'))

model.summary()
model.compile(loss='categorical_crossentropy', optimizer='adam',metrics=['accuracy'])

# Prepare model model saving directory.
save_dir = '/Users/sayantansikdar/Developer/FRUITS_MASTER/Alexnet_Model'
model_name = 'Alexnet_model_fNet_batch10_epoch50.h5'
saved_model= save_dir + '/' + model_name

# saved_model= save_dir + '/' + model_name
# saved_model_final = load_model(saved_model)
if not os.path.isdir(save_dir):
    os.makedirs(save_dir)
filepath = os.path.join(save_dir, model_name)

# Prepare callbacks for model saving and for learning rate adjustment.
checkpoint = ModelCheckpoint(filepath=filepath,
                             monitor='val_acc',
                             verbose=1,
                             save_best_only=True)

lr_scheduler = LearningRateScheduler(lr_schedule)

lr_reducer = ReduceLROnPlateau(factor=np.sqrt(0.1),
                               cooldown=0,
                               patience=5,
                               min_lr=0.5e-6)

callbacks = [checkpoint, lr_reducer, lr_scheduler]

es = EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=200) #EPOCH CHANGING HERE
mc = ModelCheckpoint(saved_model , monitor='val_accuracy', mode='max', verbose=1, save_best_only=True)


# print(type(trainImages))
# print(type(trainLabels))
# trainImages= np.asarray(trainImages).astype(np.float32)
# trainImages= np.asarray(trainImages).astype(np.float32)
# trainLabels= np.asarray(trainLabels).astype(np.float32)
model.fit(trainImages, trainLabels, batch_size= batch_size, epochs = epochs, verbose=1, validation_split=0.1846, shuffle=True, callbacks=[es, mc])

saved_model_final = load_model(saved_model)


# Score trained model.
preds = saved_model_final.predict(testImages)
print(preds)

end1= time.time()
time_taken= end1- start1

workbook = xlsxwriter.Workbook(result_excel_link)
worksheet = workbook.add_worksheet()

for i in range(0,test_img_num):
        #wb1 = load_workbook('D:/research phd/agriculture research/2nd phase of research/leaf phenotyping/label/results.xlsx')
        #ws1 = wb1.active
        #a = ws1.cell(row= i+1, column=1)
        #a.value = testLabels[i]
        #b = ws1.cell(row= i+1, column=2)
        #b.value = np.argmax(preds [i])
        #ws1.save('D:/research phd/agriculture research/2nd phase of research/leaf phenotyping/label/results.xlsx')
        name = test_img_name[i]
        worksheet.write(i+1, 0 , name)
        a = testLabels[i]
        worksheet.write(i+1, 1 , a)
        b = np.argmax(preds[i])
        worksheet.write(i+1, 2 , b)
        if a==b:
             worksheet.write(i+1, 3 , 1)
        else:
             worksheet.write(i+1, 3 , 0)

# worksheet.write(1, 4 , time_taken)

workbook.close()


11
[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1.]
Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 54, 54, 96)        34944     
                                                                 
 activation (Activation)     (None, 54, 54, 96)        0         
                                                                 
 max_pooling2d (MaxPooling2  (None, 27, 27, 96)        0         
 D)                                                              
                                                                 
 batch_normalization (Batch  (None, 27, 27, 96)        384       
 Normalization)                                                  
                                                                 
 conv2d_1 (Conv2D)           (None, 17, 17, 256)       2973952   
                                                                 
 activation_1 (

/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/keras/src/engine/training.py:3079: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


787/787 [==============================] - 522s 661ms/step - loss: 2.0118 - accuracy: 0.3642 - val_loss: 1.6549 - val_accuracy: 0.4293
Epoch 2/50
787/787 [==============================] - ETA: 0s - loss: 1.6748 - accuracy: 0.4224
Epoch 2: val_accuracy improved from 0.42929 to 0.50168, saving model to /Users/sayantansikdar/Developer/FRUITS_MASTER/Alexnet_Model/Alexnet_model_fNet_batch10_epoch50.h5
787/787 [==============================] - 518s 658ms/step - loss: 1.6748 - accuracy: 0.4224 - val_loss: 1.4350 - val_accuracy: 0.5017
Epoch 3/50
787/787 [==============================] - ETA: 0s - loss: 1.5413 - accuracy: 0.4704
Epoch 3: val_accuracy improved from 0.50168 to 0.62346, saving model to /Users/sayantansikdar/Developer/FRUITS_MASTER/Alexnet_Model/Alexnet_model_fNet_batch10_epoch50.h5
787/787 [==============================] - 531s 674ms/step - loss: 1.5413 - accuracy: 0.4704 - val_loss: 1.1675 - val_accuracy: 0.6235
Epoch 4/50
787/787 [==============================] - ETA: 0s -

In [2]:
# -*- coding: utf-8 -*-
"""Alexnet_code.ipynb

Automatically generated by Colaboratory.

Original file is located at
    https://colab.research.google.com/drive/1ST_yrafE2p_4JHcbvSpBBYMW1lWIw5hD
"""

from __future__ import print_function
import keras
import pandas as pd
from keras.layers import Dense, Conv2D, BatchNormalization, Activation
from keras.layers import AveragePooling2D, Input, Flatten
from keras.optimizers import Adam
from keras.callbacks import ModelCheckpoint, LearningRateScheduler
from keras.callbacks import ReduceLROnPlateau
from keras.callbacks import EarlyStopping
from keras.callbacks import ModelCheckpoint
from keras.preprocessing.image import ImageDataGenerator
from keras.regularizers import l2
from keras import backend as K
from keras.models import Model
from keras.models import load_model
from keras.utils import to_categorical
import numpy as np
import os
import time
from openpyxl import load_workbook
from keras.models import Sequential
from keras.layers import Dense, Activation, Dropout, Flatten,Conv2D, MaxPooling2D
from keras.layers import BatchNormalization
from PIL import Image
import tensorflow as tf

import xlsxwriter
import cv2
from random import shuffle




start1= time.time()

# Training parameters
batch_size = 10  # orig paper trained all networks with batch_size=128
epochs = 75
data_augmentation = False
num_classes = 12
test_img_num = 2083
result_excel_link = '/Users/sayantansikdar/Developer/FRUITS_MASTER/Alexnet_Result/fruit_batch10_epoch75.xlsx'
Train_DIR= '/Users/sayantansikdar/Developer/FRUITS_MASTER/FINAL DATA_Fruit/TRAIN'
TEST_DIR= '/Users/sayantansikdar/Developer/FRUITS_MASTER/FINAL DATA_Fruit/TEST'
label_link= '/Users/sayantansikdar/Developer/FRUITS_MASTER/FINAL_encoded_file.xlsx'

# Subtracting pixel mean improves accuracy
subtract_pixel_mean = True

# Model parameter
# ----------------------------------------------------------------------------
#           |      | 200-epoch | Orig Paper| 200-epoch | Orig Paper| sec/epoch
# Model     |  n   | ResNet v1 | ResNet v1 | ResNet v2 | ResNet v2 | GTX1080Ti
#           |v1(v2)| %Accuracy | %Accuracy | %Accuracy | %Accuracy | v1 (v2)
# ----------------------------------------------------------------------------
# ResNet20  | 3 (2)| 92.16     | 91.25     | -----     | -----     | 35 (---)
# ResNet32  | 5(NA)| 92.46     | 92.49     | NA        | NA        | 50 ( NA)
# ResNet44  | 7(NA)| 92.50     | 92.83     | NA        | NA        | 70 ( NA)
# ResNet56  | 9 (6)| 92.71     | 93.03     | 93.01     | NA        | 90 (100)
# ResNet110 |18(12)| 92.65     | 93.39+-.16| 93.15     | 93.63     | 165(180)
# ResNet164 |27(18)| -----     | 94.07     | -----     | 94.54     | ---(---)
# ResNet1001| (111)| -----     | 92.39     | -----     | 95.08+-.14| ---(---)
# ---------------------------------------------------------------------------


# Model version
# Orig paper: version = 1 (ResNet v1), Improved ResNet: version = 2 (ResNet v2)
version = 2
n = 2
# Computed depth from supplied model parameter n
if version == 1:
    depth = n * 6 + 2
elif version == 2:
    depth = n * 9 + 2

# Model name, depth and version
model_type = 'AlexNet%dv%d' % (depth, version)

# Load the CIFAR10 data.
##(x_train, y_train), (x_test, y_test) = cifar10.load_data()



def label_img(name):
    # print(name)
    filename1 = os.fsdecode(name)
    # print(filename1)
    s=[]
    s= filename1.split('_')
    # print(s)
    # print(int(s[0]))
    #word_label = name.split('-')[0]
    #if word_label == 'golden_retriever': return np.array([1, 0])
    #elif word_label == 'shetland_sheepdog' : return np.array([0, 1])
    wb = load_workbook(label_link)
    ws = wb.active
    #for cell in ws.columns[1]:  #here column 9 is value is what i am testing which is Column X of my example.
    for x in range(2, 18454):
        #for y in range(1,):
        cell= ws.cell(row=x, column=1)
        # print(cell.value)
        # print(cell.value)
        if cell.value == int(s[0]) :
               #print ws.cell(column=12).value
                #a= ws.columns(row).value
                #b= ws.columns(col).value
                cell2= ws.cell(row=x, column=2)

                label1= cell2.value
                # print(label1)
                break
        #else:
                #print('hi')
    return label1

def load_training_data():
    train_data = []
    for img in os.listdir(Train_DIR):
        #print(img)
        temp2 = img.replace('.png','')
        label = label_img(temp2)
        #print(label)
        path = os.path.join(Train_DIR, img)
        #if "DS_Store" not in path:
        #img = Image.open(path)
        # print(path)
        # print(label)
        img2= cv2.imread(path)
        # cv2_imshow(img2)
        #img = img.convert('L')
        # img = img.resize((IMG_SIZE, IMG_SIZE, 3), Image.ANTIALIAS)
        #path3= 'D:/research phd/agriculture research/2nd phase of research/leaf phenotyping/training_441_segmented_polar_A1_A2_A4'
        resized_img1 = cv2.resize(img2, (224,224), interpolation = cv2.INTER_AREA)
        #cv2.imwrite(path3+ '/' + temp2 , resized_img1 )
        train_data.append([np.array(resized_img1), label])

    shuffle(train_data)
    return train_data


train_data = load_training_data()

# s = [i[1] for i in train_data]
# print(train_data)

# df2 = pd.DataFrame(np.array(df2),columns=['Label'])
# df2['Label'] = pd.Categorical(df2.Label)
IMG_SIZE = 224
# trainImages = np.array([i[0] for i in train_data],dtype=object) #.reshape(-1, IMG_SIZE, IMG_SIZE, 1)
trainImages = np.array([i[0] for i in train_data])
# df = pd.read_csv()
# trainLabels = tf.convert_to_tensor(np.array([i[1] for i in train_data]).astype(np.int_))
trainLabels = np.array([i[1] for i in train_data])
print(trainLabels[2]) #successful till here
#print(trainLabels)
# trainLabels = to_categorical(trainLabels)                                                          #problem here
trainLabels = keras.utils.to_categorical(trainLabels, num_classes)
print(trainLabels[2])



def load_test_data():
    test_data = []
    for img in os.listdir(TEST_DIR):
        temp1 = img.replace('.png','')
        # print(temp1)
        label = label_img(temp1)
        path = os.path.join(TEST_DIR, img)
        # temp1 = img
        #if "DS_Store" not in path:
        #img = Image.open(path)
        img1= cv2.imread(path)
        #img = img.convert('L')
        #img = img.resize((IMG_SIZE, IMG_SIZE, 3), Image.ANTIALIAS)
        resized_img2 = cv2.resize(img1, (224,224), interpolation = cv2.INTER_AREA)
        #path2= 'D:/research phd/agriculture research/2nd phase of research/leaf phenotyping/testing folder_segmented_resized'
        #cv2.imwrite(path2+ '/' + temp1 , resized_img2 )
        test_data.append([np.array(resized_img2), label,temp1])
        # print(label)

    shuffle(test_data)
    return test_data


test_data = load_test_data()
#plt.imshow(test_data[10][0], cmap = 'gist_gray')


# # convert it to tensorflow              ########################
# tensor1 = tf.convert_to_tensor(numpy_array)
# print(tensor1)

# testImages = np.array([i[0] for i in test_data],dtype=object) #.reshape(-1, IMG_SIZE, IMG_SIZE, 1)
# testLabels = tf.convert_to_tensor(np.array([i[1] for i in test_data]).astype(np.int_))

testImages = np.array([i[0] for i in test_data])
testLabels = np.array([i[1] for i in test_data])
test_img_name = np.array([i[2] for i in test_data])


#testLabels = keras.utils.to_categorical(testLabels, num_classes)

# Input image dimensions.
input_shape = trainImages.shape[1:]

# Normalize data.
##trainImages = trainImages.astype('float32') / 255
##testImages = testImages.astype('float32') / 255

# If subtract pixel mean is enabled
##if subtract_pixel_mean:
    ##trainImages_mean = np.mean(trainImages, axis=0)
    ##trainImages -= trainImages_mean
    ##testImages -= trainImages_mean

#print('x_train shape:', x_train.shape)
#print(x_train.shape[0], 'train samples')
#print(x_test.shape[0], 'test samples')
#print('y_train shape:', y_train.shape)

# Convert class vectors to binary class matrices.
#y_train = keras.utils.to_categorical(y_train, num_classes)
#y_test = keras.utils.to_categorical(y_test, num_classes)

def lr_schedule(epoch):
    """Learning Rate Schedule

    Learning rate is scheduled to be reduced after 80, 120, 160, 180 epochs.
    Called automatically every epoch as part of callbacks during training.

    # Arguments
        epoch (int): The number of epochs

    # Returns
        lr (float32): learning rate
    """
    lr = 1e-3
    if epoch > 180:
        lr *= 0.5e-3
    elif epoch > 160:
        lr *= 1e-3
    elif epoch > 120:
        lr *= 1e-2
    elif epoch > 80:
        lr *= 1e-1
    print('Learning rate: ', lr)
    return lr


model = Sequential()

# 1st Convolutional Layer
model.add(Conv2D(filters=96, input_shape=(224,224,3), kernel_size=(11,11),strides=(4,4), padding='valid'))
model.add(Activation('relu'))
# Pooling
model.add(MaxPooling2D(pool_size=(2,2), strides=(2,2), padding='valid'))
# Batch Normalisation before passing it to the next layer
model.add(BatchNormalization())

# 2nd Convolutional Layer
model.add(Conv2D(filters=256, kernel_size=(11,11), strides=(1,1), padding='valid'))
model.add(Activation('relu'))
# Pooling
model.add(MaxPooling2D(pool_size=(2,2), strides=(2,2), padding='valid'))
# Batch Normalisation
model.add(BatchNormalization())

# 3rd Convolutional Layer
model.add(Conv2D(filters=384, kernel_size=(3,3), strides=(1,1), padding='valid'))
model.add(Activation('relu'))
# Batch Normalisation
model.add(BatchNormalization())

# 4th Convolutional Layer
model.add(Conv2D(filters=384, kernel_size=(3,3), strides=(1,1), padding='valid'))
model.add(Activation('relu'))
# Batch Normalisation
model.add(BatchNormalization())

# 5th Convolutional Layer
model.add(Conv2D(filters=256, kernel_size=(3,3), strides=(1,1), padding='valid'))
model.add(Activation('relu'))
# Pooling
model.add(MaxPooling2D(pool_size=(2,2), strides=(2,2), padding='valid'))
# Batch Normalisation
model.add(BatchNormalization())

# Passing it to a dense layer
model.add(Flatten())
# 1st Dense Layer
model.add(Dense(4096, input_shape=(224*224*3,)))
model.add(Activation('relu'))
# Add Dropout to prevent overfitting
model.add(Dropout(0.4))
# Batch Normalisation
model.add(BatchNormalization())

# 2nd Dense Layer
model.add(Dense(4096))
model.add(Activation('relu'))
# Add Dropout
model.add(Dropout(0.4))
# Batch Normalisation
model.add(BatchNormalization())

# 3rd Dense Layer
model.add(Dense(1000))
model.add(Activation('relu'))
# Add Dropout
model.add(Dropout(0.4))
# Batch Normalisation
model.add(BatchNormalization())

# 4th Dense Layer
model.add(Dense(100))
model.add(Activation('relu'))
# Add Dropout
model.add(Dropout(0.4))
# Batch Normalisation
model.add(BatchNormalization())


# Output Layer
model.add(Dense(num_classes))
model.add(Activation('softmax'))

model.summary()
model.compile(loss='categorical_crossentropy', optimizer='adam',metrics=['accuracy'])

# Prepare model model saving directory.
save_dir = '/Users/sayantansikdar/Developer/FRUITS_MASTER/Alexnet_Model'
model_name = 'Alexnet_model_fNet_batch10_epoch75.h5'
saved_model= save_dir + '/' + model_name

# saved_model= save_dir + '/' + model_name
# saved_model_final = load_model(saved_model)
if not os.path.isdir(save_dir):
    os.makedirs(save_dir)
filepath = os.path.join(save_dir, model_name)

# Prepare callbacks for model saving and for learning rate adjustment.
checkpoint = ModelCheckpoint(filepath=filepath,
                             monitor='val_acc',
                             verbose=1,
                             save_best_only=True)

lr_scheduler = LearningRateScheduler(lr_schedule)

lr_reducer = ReduceLROnPlateau(factor=np.sqrt(0.1),
                               cooldown=0,
                               patience=5,
                               min_lr=0.5e-6)

callbacks = [checkpoint, lr_reducer, lr_scheduler]

es = EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=200) #EPOCH CHANGING HERE
mc = ModelCheckpoint(saved_model , monitor='val_accuracy', mode='max', verbose=1, save_best_only=True)


# print(type(trainImages))
# print(type(trainLabels))
# trainImages= np.asarray(trainImages).astype(np.float32)
# trainImages= np.asarray(trainImages).astype(np.float32)
# trainLabels= np.asarray(trainLabels).astype(np.float32)
model.fit(trainImages, trainLabels, batch_size= batch_size, epochs = epochs, verbose=1, validation_split=0.1846, shuffle=True, callbacks=[es, mc])

saved_model_final = load_model(saved_model)


# Score trained model.
preds = saved_model_final.predict(testImages)
print(preds)

end1= time.time()
time_taken= end1- start1

workbook = xlsxwriter.Workbook(result_excel_link)
worksheet = workbook.add_worksheet()

for i in range(0,test_img_num):
        #wb1 = load_workbook('D:/research phd/agriculture research/2nd phase of research/leaf phenotyping/label/results.xlsx')
        #ws1 = wb1.active
        #a = ws1.cell(row= i+1, column=1)
        #a.value = testLabels[i]
        #b = ws1.cell(row= i+1, column=2)
        #b.value = np.argmax(preds [i])
        #ws1.save('D:/research phd/agriculture research/2nd phase of research/leaf phenotyping/label/results.xlsx')
        name = test_img_name[i]
        worksheet.write(i+1, 0 , name)
        a = testLabels[i]
        worksheet.write(i+1, 1 , a)
        b = np.argmax(preds[i])
        worksheet.write(i+1, 2 , b)
        if a==b:
             worksheet.write(i+1, 3 , 1)
        else:
             worksheet.write(i+1, 3 , 0)

# worksheet.write(1, 4 , time_taken)

workbook.close()


4
[0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0.]
Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_5 (Conv2D)           (None, 54, 54, 96)        34944     
                                                                 
 activation_10 (Activation)  (None, 54, 54, 96)        0         
                                                                 
 max_pooling2d_3 (MaxPoolin  (None, 27, 27, 96)        0         
 g2D)                                                            
                                                                 
 batch_normalization_9 (Bat  (None, 27, 27, 96)        384       
 chNormalization)                                                
                                                                 
 conv2d_6 (Conv2D)           (None, 17, 17, 256)       2973952   
                                                                 
 activation_11

In [3]:
# -*- coding: utf-8 -*-
"""Alexnet_code.ipynb

Automatically generated by Colaboratory.

Original file is located at
    https://colab.research.google.com/drive/1ST_yrafE2p_4JHcbvSpBBYMW1lWIw5hD
"""

from __future__ import print_function
import keras
import pandas as pd
from keras.layers import Dense, Conv2D, BatchNormalization, Activation
from keras.layers import AveragePooling2D, Input, Flatten
from keras.optimizers import Adam
from keras.callbacks import ModelCheckpoint, LearningRateScheduler
from keras.callbacks import ReduceLROnPlateau
from keras.callbacks import EarlyStopping
from keras.callbacks import ModelCheckpoint
from keras.preprocessing.image import ImageDataGenerator
from keras.regularizers import l2
from keras import backend as K
from keras.models import Model
from keras.models import load_model
from keras.utils import to_categorical
import numpy as np
import os
import time
from openpyxl import load_workbook
from keras.models import Sequential
from keras.layers import Dense, Activation, Dropout, Flatten,Conv2D, MaxPooling2D
from keras.layers import BatchNormalization
from PIL import Image
import tensorflow as tf

import xlsxwriter
import cv2
from random import shuffle




start1= time.time()

# Training parameters
batch_size = 10  # orig paper trained all networks with batch_size=128
epochs = 100
data_augmentation = False
num_classes = 12
test_img_num = 2083
result_excel_link = '/Users/sayantansikdar/Developer/FRUITS_MASTER/Alexnet_Result/fruit_batch10_epoch100.xlsx'
Train_DIR= '/Users/sayantansikdar/Developer/FRUITS_MASTER/FINAL DATA_Fruit/TRAIN'
TEST_DIR= '/Users/sayantansikdar/Developer/FRUITS_MASTER/FINAL DATA_Fruit/TEST'
label_link= '/Users/sayantansikdar/Developer/FRUITS_MASTER/FINAL_encoded_file.xlsx'

# Subtracting pixel mean improves accuracy
subtract_pixel_mean = True

# Model parameter
# ----------------------------------------------------------------------------
#           |      | 200-epoch | Orig Paper| 200-epoch | Orig Paper| sec/epoch
# Model     |  n   | ResNet v1 | ResNet v1 | ResNet v2 | ResNet v2 | GTX1080Ti
#           |v1(v2)| %Accuracy | %Accuracy | %Accuracy | %Accuracy | v1 (v2)
# ----------------------------------------------------------------------------
# ResNet20  | 3 (2)| 92.16     | 91.25     | -----     | -----     | 35 (---)
# ResNet32  | 5(NA)| 92.46     | 92.49     | NA        | NA        | 50 ( NA)
# ResNet44  | 7(NA)| 92.50     | 92.83     | NA        | NA        | 70 ( NA)
# ResNet56  | 9 (6)| 92.71     | 93.03     | 93.01     | NA        | 90 (100)
# ResNet110 |18(12)| 92.65     | 93.39+-.16| 93.15     | 93.63     | 165(180)
# ResNet164 |27(18)| -----     | 94.07     | -----     | 94.54     | ---(---)
# ResNet1001| (111)| -----     | 92.39     | -----     | 95.08+-.14| ---(---)
# ---------------------------------------------------------------------------


# Model version
# Orig paper: version = 1 (ResNet v1), Improved ResNet: version = 2 (ResNet v2)
version = 2
n = 2
# Computed depth from supplied model parameter n
if version == 1:
    depth = n * 6 + 2
elif version == 2:
    depth = n * 9 + 2

# Model name, depth and version
model_type = 'AlexNet%dv%d' % (depth, version)

# Load the CIFAR10 data.
##(x_train, y_train), (x_test, y_test) = cifar10.load_data()



def label_img(name):
    # print(name)
    filename1 = os.fsdecode(name)
    # print(filename1)
    s=[]
    s= filename1.split('_')
    # print(s)
    # print(int(s[0]))
    #word_label = name.split('-')[0]
    #if word_label == 'golden_retriever': return np.array([1, 0])
    #elif word_label == 'shetland_sheepdog' : return np.array([0, 1])
    wb = load_workbook(label_link)
    ws = wb.active
    #for cell in ws.columns[1]:  #here column 9 is value is what i am testing which is Column X of my example.
    for x in range(2, 18454):
        #for y in range(1,):
        cell= ws.cell(row=x, column=1)
        # print(cell.value)
        # print(cell.value)
        if cell.value == int(s[0]) :
               #print ws.cell(column=12).value
                #a= ws.columns(row).value
                #b= ws.columns(col).value
                cell2= ws.cell(row=x, column=2)

                label1= cell2.value
                # print(label1)
                break
        #else:
                #print('hi')
    return label1

def load_training_data():
    train_data = []
    for img in os.listdir(Train_DIR):
        #print(img)
        temp2 = img.replace('.png','')
        label = label_img(temp2)
        #print(label)
        path = os.path.join(Train_DIR, img)
        #if "DS_Store" not in path:
        #img = Image.open(path)
        # print(path)
        # print(label)
        img2= cv2.imread(path)
        # cv2_imshow(img2)
        #img = img.convert('L')
        # img = img.resize((IMG_SIZE, IMG_SIZE, 3), Image.ANTIALIAS)
        #path3= 'D:/research phd/agriculture research/2nd phase of research/leaf phenotyping/training_441_segmented_polar_A1_A2_A4'
        resized_img1 = cv2.resize(img2, (224,224), interpolation = cv2.INTER_AREA)
        #cv2.imwrite(path3+ '/' + temp2 , resized_img1 )
        train_data.append([np.array(resized_img1), label])

    shuffle(train_data)
    return train_data


train_data = load_training_data()

# s = [i[1] for i in train_data]
# print(train_data)

# df2 = pd.DataFrame(np.array(df2),columns=['Label'])
# df2['Label'] = pd.Categorical(df2.Label)
IMG_SIZE = 224
# trainImages = np.array([i[0] for i in train_data],dtype=object) #.reshape(-1, IMG_SIZE, IMG_SIZE, 1)
trainImages = np.array([i[0] for i in train_data])
# df = pd.read_csv()
# trainLabels = tf.convert_to_tensor(np.array([i[1] for i in train_data]).astype(np.int_))
trainLabels = np.array([i[1] for i in train_data])
print(trainLabels[2]) #successful till here
#print(trainLabels)
# trainLabels = to_categorical(trainLabels)                                                          #problem here
trainLabels = keras.utils.to_categorical(trainLabels, num_classes)
print(trainLabels[2])



def load_test_data():
    test_data = []
    for img in os.listdir(TEST_DIR):
        temp1 = img.replace('.png','')
        # print(temp1)
        label = label_img(temp1)
        path = os.path.join(TEST_DIR, img)
        # temp1 = img
        #if "DS_Store" not in path:
        #img = Image.open(path)
        img1= cv2.imread(path)
        #img = img.convert('L')
        #img = img.resize((IMG_SIZE, IMG_SIZE, 3), Image.ANTIALIAS)
        resized_img2 = cv2.resize(img1, (224,224), interpolation = cv2.INTER_AREA)
        #path2= 'D:/research phd/agriculture research/2nd phase of research/leaf phenotyping/testing folder_segmented_resized'
        #cv2.imwrite(path2+ '/' + temp1 , resized_img2 )
        test_data.append([np.array(resized_img2), label,temp1])
        # print(label)

    shuffle(test_data)
    return test_data


test_data = load_test_data()
#plt.imshow(test_data[10][0], cmap = 'gist_gray')


# # convert it to tensorflow              ########################
# tensor1 = tf.convert_to_tensor(numpy_array)
# print(tensor1)

# testImages = np.array([i[0] for i in test_data],dtype=object) #.reshape(-1, IMG_SIZE, IMG_SIZE, 1)
# testLabels = tf.convert_to_tensor(np.array([i[1] for i in test_data]).astype(np.int_))

testImages = np.array([i[0] for i in test_data])
testLabels = np.array([i[1] for i in test_data])
test_img_name = np.array([i[2] for i in test_data])


#testLabels = keras.utils.to_categorical(testLabels, num_classes)

# Input image dimensions.
input_shape = trainImages.shape[1:]

# Normalize data.
##trainImages = trainImages.astype('float32') / 255
##testImages = testImages.astype('float32') / 255

# If subtract pixel mean is enabled
##if subtract_pixel_mean:
    ##trainImages_mean = np.mean(trainImages, axis=0)
    ##trainImages -= trainImages_mean
    ##testImages -= trainImages_mean

#print('x_train shape:', x_train.shape)
#print(x_train.shape[0], 'train samples')
#print(x_test.shape[0], 'test samples')
#print('y_train shape:', y_train.shape)

# Convert class vectors to binary class matrices.
#y_train = keras.utils.to_categorical(y_train, num_classes)
#y_test = keras.utils.to_categorical(y_test, num_classes)

def lr_schedule(epoch):
    """Learning Rate Schedule

    Learning rate is scheduled to be reduced after 80, 120, 160, 180 epochs.
    Called automatically every epoch as part of callbacks during training.

    # Arguments
        epoch (int): The number of epochs

    # Returns
        lr (float32): learning rate
    """
    lr = 1e-3
    if epoch > 180:
        lr *= 0.5e-3
    elif epoch > 160:
        lr *= 1e-3
    elif epoch > 120:
        lr *= 1e-2
    elif epoch > 80:
        lr *= 1e-1
    print('Learning rate: ', lr)
    return lr


model = Sequential()

# 1st Convolutional Layer
model.add(Conv2D(filters=96, input_shape=(224,224,3), kernel_size=(11,11),strides=(4,4), padding='valid'))
model.add(Activation('relu'))
# Pooling
model.add(MaxPooling2D(pool_size=(2,2), strides=(2,2), padding='valid'))
# Batch Normalisation before passing it to the next layer
model.add(BatchNormalization())

# 2nd Convolutional Layer
model.add(Conv2D(filters=256, kernel_size=(11,11), strides=(1,1), padding='valid'))
model.add(Activation('relu'))
# Pooling
model.add(MaxPooling2D(pool_size=(2,2), strides=(2,2), padding='valid'))
# Batch Normalisation
model.add(BatchNormalization())

# 3rd Convolutional Layer
model.add(Conv2D(filters=384, kernel_size=(3,3), strides=(1,1), padding='valid'))
model.add(Activation('relu'))
# Batch Normalisation
model.add(BatchNormalization())

# 4th Convolutional Layer
model.add(Conv2D(filters=384, kernel_size=(3,3), strides=(1,1), padding='valid'))
model.add(Activation('relu'))
# Batch Normalisation
model.add(BatchNormalization())

# 5th Convolutional Layer
model.add(Conv2D(filters=256, kernel_size=(3,3), strides=(1,1), padding='valid'))
model.add(Activation('relu'))
# Pooling
model.add(MaxPooling2D(pool_size=(2,2), strides=(2,2), padding='valid'))
# Batch Normalisation
model.add(BatchNormalization())

# Passing it to a dense layer
model.add(Flatten())
# 1st Dense Layer
model.add(Dense(4096, input_shape=(224*224*3,)))
model.add(Activation('relu'))
# Add Dropout to prevent overfitting
model.add(Dropout(0.4))
# Batch Normalisation
model.add(BatchNormalization())

# 2nd Dense Layer
model.add(Dense(4096))
model.add(Activation('relu'))
# Add Dropout
model.add(Dropout(0.4))
# Batch Normalisation
model.add(BatchNormalization())

# 3rd Dense Layer
model.add(Dense(1000))
model.add(Activation('relu'))
# Add Dropout
model.add(Dropout(0.4))
# Batch Normalisation
model.add(BatchNormalization())

# 4th Dense Layer
model.add(Dense(100))
model.add(Activation('relu'))
# Add Dropout
model.add(Dropout(0.4))
# Batch Normalisation
model.add(BatchNormalization())


# Output Layer
model.add(Dense(num_classes))
model.add(Activation('softmax'))

model.summary()
model.compile(loss='categorical_crossentropy', optimizer='adam',metrics=['accuracy'])

# Prepare model model saving directory.
save_dir = '/Users/sayantansikdar/Developer/FRUITS_MASTER/Alexnet_Model'
model_name = 'Alexnet_model_fNet_batch10_epoch100.h5'
saved_model= save_dir + '/' + model_name

# saved_model= save_dir + '/' + model_name
# saved_model_final = load_model(saved_model)
if not os.path.isdir(save_dir):
    os.makedirs(save_dir)
filepath = os.path.join(save_dir, model_name)

# Prepare callbacks for model saving and for learning rate adjustment.
checkpoint = ModelCheckpoint(filepath=filepath,
                             monitor='val_acc',
                             verbose=1,
                             save_best_only=True)

lr_scheduler = LearningRateScheduler(lr_schedule)

lr_reducer = ReduceLROnPlateau(factor=np.sqrt(0.1),
                               cooldown=0,
                               patience=5,
                               min_lr=0.5e-6)

callbacks = [checkpoint, lr_reducer, lr_scheduler]

es = EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=200) #EPOCH CHANGING HERE
mc = ModelCheckpoint(saved_model , monitor='val_accuracy', mode='max', verbose=1, save_best_only=True)


# print(type(trainImages))
# print(type(trainLabels))
# trainImages= np.asarray(trainImages).astype(np.float32)
# trainImages= np.asarray(trainImages).astype(np.float32)
# trainLabels= np.asarray(trainLabels).astype(np.float32)
model.fit(trainImages, trainLabels, batch_size= batch_size, epochs = epochs, verbose=1, validation_split=0.1846, shuffle=True, callbacks=[es, mc])

saved_model_final = load_model(saved_model)


# Score trained model.
preds = saved_model_final.predict(testImages)
print(preds)

end1= time.time()
time_taken= end1- start1

workbook = xlsxwriter.Workbook(result_excel_link)
worksheet = workbook.add_worksheet()

for i in range(0,test_img_num):
        #wb1 = load_workbook('D:/research phd/agriculture research/2nd phase of research/leaf phenotyping/label/results.xlsx')
        #ws1 = wb1.active
        #a = ws1.cell(row= i+1, column=1)
        #a.value = testLabels[i]
        #b = ws1.cell(row= i+1, column=2)
        #b.value = np.argmax(preds [i])
        #ws1.save('D:/research phd/agriculture research/2nd phase of research/leaf phenotyping/label/results.xlsx')
        name = test_img_name[i]
        worksheet.write(i+1, 0 , name)
        a = testLabels[i]
        worksheet.write(i+1, 1 , a)
        b = np.argmax(preds[i])
        worksheet.write(i+1, 2 , b)
        if a==b:
             worksheet.write(i+1, 3 , 1)
        else:
             worksheet.write(i+1, 3 , 0)

# worksheet.write(1, 4 , time_taken)

workbook.close()


11
[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1.]
Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_10 (Conv2D)          (None, 54, 54, 96)        34944     
                                                                 
 activation_20 (Activation)  (None, 54, 54, 96)        0         
                                                                 
 max_pooling2d_6 (MaxPoolin  (None, 27, 27, 96)        0         
 g2D)                                                            
                                                                 
 batch_normalization_18 (Ba  (None, 27, 27, 96)        384       
 tchNormalization)                                               
                                                                 
 conv2d_11 (Conv2D)          (None, 17, 17, 256)       2973952   
                                                                 
 activation_2

Epoch 150

In [4]:
# -*- coding: utf-8 -*-
"""Alexnet_code.ipynb

Automatically generated by Colaboratory.

Original file is located at
    https://colab.research.google.com/drive/1ST_yrafE2p_4JHcbvSpBBYMW1lWIw5hD
"""

from __future__ import print_function
import keras
import pandas as pd
from keras.layers import Dense, Conv2D, BatchNormalization, Activation
from keras.layers import AveragePooling2D, Input, Flatten
from keras.optimizers import Adam
from keras.callbacks import ModelCheckpoint, LearningRateScheduler
from keras.callbacks import ReduceLROnPlateau
from keras.callbacks import EarlyStopping
from keras.callbacks import ModelCheckpoint
from keras.preprocessing.image import ImageDataGenerator
from keras.regularizers import l2
from keras import backend as K
from keras.models import Model
from keras.models import load_model
from keras.utils import to_categorical
import numpy as np
import os
import time
from openpyxl import load_workbook
from keras.models import Sequential
from keras.layers import Dense, Activation, Dropout, Flatten,Conv2D, MaxPooling2D
from keras.layers import BatchNormalization
from PIL import Image
import tensorflow as tf

import xlsxwriter
import cv2
from random import shuffle




start1= time.time()

# Training parameters
batch_size = 10  # orig paper trained all networks with batch_size=128
epochs = 150
data_augmentation = False
num_classes = 12
test_img_num = 2083
result_excel_link = '/Users/sayantansikdar/Developer/FRUITS_MASTER/Alexnet_Result/fruit_batch10_epoch150.xlsx'
Train_DIR= '/Users/sayantansikdar/Developer/FRUITS_MASTER/FINAL DATA_Fruit/TRAIN'
TEST_DIR= '/Users/sayantansikdar/Developer/FRUITS_MASTER/FINAL DATA_Fruit/TEST'
label_link= '/Users/sayantansikdar/Developer/FRUITS_MASTER/FINAL_encoded_file.xlsx'

# Subtracting pixel mean improves accuracy
subtract_pixel_mean = True

# Model parameter
# ----------------------------------------------------------------------------
#           |      | 200-epoch | Orig Paper| 200-epoch | Orig Paper| sec/epoch
# Model     |  n   | ResNet v1 | ResNet v1 | ResNet v2 | ResNet v2 | GTX1080Ti
#           |v1(v2)| %Accuracy | %Accuracy | %Accuracy | %Accuracy | v1 (v2)
# ----------------------------------------------------------------------------
# ResNet20  | 3 (2)| 92.16     | 91.25     | -----     | -----     | 35 (---)
# ResNet32  | 5(NA)| 92.46     | 92.49     | NA        | NA        | 50 ( NA)
# ResNet44  | 7(NA)| 92.50     | 92.83     | NA        | NA        | 70 ( NA)
# ResNet56  | 9 (6)| 92.71     | 93.03     | 93.01     | NA        | 90 (100)
# ResNet110 |18(12)| 92.65     | 93.39+-.16| 93.15     | 93.63     | 165(180)
# ResNet164 |27(18)| -----     | 94.07     | -----     | 94.54     | ---(---)
# ResNet1001| (111)| -----     | 92.39     | -----     | 95.08+-.14| ---(---)
# ---------------------------------------------------------------------------


# Model version
# Orig paper: version = 1 (ResNet v1), Improved ResNet: version = 2 (ResNet v2)
version = 2
n = 2
# Computed depth from supplied model parameter n
if version == 1:
    depth = n * 6 + 2
elif version == 2:
    depth = n * 9 + 2

# Model name, depth and version
model_type = 'AlexNet%dv%d' % (depth, version)

# Load the CIFAR10 data.
##(x_train, y_train), (x_test, y_test) = cifar10.load_data()



def label_img(name):
    # print(name)
    filename1 = os.fsdecode(name)
    # print(filename1)
    s=[]
    s= filename1.split('_')
    # print(s)
    # print(int(s[0]))
    #word_label = name.split('-')[0]
    #if word_label == 'golden_retriever': return np.array([1, 0])
    #elif word_label == 'shetland_sheepdog' : return np.array([0, 1])
    wb = load_workbook(label_link)
    ws = wb.active
    #for cell in ws.columns[1]:  #here column 9 is value is what i am testing which is Column X of my example.
    for x in range(2, 18454):
        #for y in range(1,):
        cell= ws.cell(row=x, column=1)
        # print(cell.value)
        # print(cell.value)
        if cell.value == int(s[0]) :
               #print ws.cell(column=12).value
                #a= ws.columns(row).value
                #b= ws.columns(col).value
                cell2= ws.cell(row=x, column=2)

                label1= cell2.value
                # print(label1)
                break
        #else:
                #print('hi')
    return label1

def load_training_data():
    train_data = []
    for img in os.listdir(Train_DIR):
        #print(img)
        temp2 = img.replace('.png','')
        label = label_img(temp2)
        #print(label)
        path = os.path.join(Train_DIR, img)
        #if "DS_Store" not in path:
        #img = Image.open(path)
        # print(path)
        # print(label)
        img2= cv2.imread(path)
        # cv2_imshow(img2)
        #img = img.convert('L')
        # img = img.resize((IMG_SIZE, IMG_SIZE, 3), Image.ANTIALIAS)
        #path3= 'D:/research phd/agriculture research/2nd phase of research/leaf phenotyping/training_441_segmented_polar_A1_A2_A4'
        resized_img1 = cv2.resize(img2, (224,224), interpolation = cv2.INTER_AREA)
        #cv2.imwrite(path3+ '/' + temp2 , resized_img1 )
        train_data.append([np.array(resized_img1), label])

    shuffle(train_data)
    return train_data


train_data = load_training_data()

# s = [i[1] for i in train_data]
# print(train_data)

# df2 = pd.DataFrame(np.array(df2),columns=['Label'])
# df2['Label'] = pd.Categorical(df2.Label)
IMG_SIZE = 224
# trainImages = np.array([i[0] for i in train_data],dtype=object) #.reshape(-1, IMG_SIZE, IMG_SIZE, 1)
trainImages = np.array([i[0] for i in train_data])
# df = pd.read_csv()
# trainLabels = tf.convert_to_tensor(np.array([i[1] for i in train_data]).astype(np.int_))
trainLabels = np.array([i[1] for i in train_data])
print(trainLabels[2]) #successful till here
#print(trainLabels)
# trainLabels = to_categorical(trainLabels)                                                          #problem here
trainLabels = keras.utils.to_categorical(trainLabels, num_classes)
print(trainLabels[2])



def load_test_data():
    test_data = []
    for img in os.listdir(TEST_DIR):
        temp1 = img.replace('.png','')
        # print(temp1)
        label = label_img(temp1)
        path = os.path.join(TEST_DIR, img)
        # temp1 = img
        #if "DS_Store" not in path:
        #img = Image.open(path)
        img1= cv2.imread(path)
        #img = img.convert('L')
        #img = img.resize((IMG_SIZE, IMG_SIZE, 3), Image.ANTIALIAS)
        resized_img2 = cv2.resize(img1, (224,224), interpolation = cv2.INTER_AREA)
        #path2= 'D:/research phd/agriculture research/2nd phase of research/leaf phenotyping/testing folder_segmented_resized'
        #cv2.imwrite(path2+ '/' + temp1 , resized_img2 )
        test_data.append([np.array(resized_img2), label,temp1])
        # print(label)

    shuffle(test_data)
    return test_data


test_data = load_test_data()
#plt.imshow(test_data[10][0], cmap = 'gist_gray')


# # convert it to tensorflow              ########################
# tensor1 = tf.convert_to_tensor(numpy_array)
# print(tensor1)

# testImages = np.array([i[0] for i in test_data],dtype=object) #.reshape(-1, IMG_SIZE, IMG_SIZE, 1)
# testLabels = tf.convert_to_tensor(np.array([i[1] for i in test_data]).astype(np.int_))

testImages = np.array([i[0] for i in test_data])
testLabels = np.array([i[1] for i in test_data])
test_img_name = np.array([i[2] for i in test_data])


#testLabels = keras.utils.to_categorical(testLabels, num_classes)

# Input image dimensions.
input_shape = trainImages.shape[1:]

# Normalize data.
##trainImages = trainImages.astype('float32') / 255
##testImages = testImages.astype('float32') / 255

# If subtract pixel mean is enabled
##if subtract_pixel_mean:
    ##trainImages_mean = np.mean(trainImages, axis=0)
    ##trainImages -= trainImages_mean
    ##testImages -= trainImages_mean

#print('x_train shape:', x_train.shape)
#print(x_train.shape[0], 'train samples')
#print(x_test.shape[0], 'test samples')
#print('y_train shape:', y_train.shape)

# Convert class vectors to binary class matrices.
#y_train = keras.utils.to_categorical(y_train, num_classes)
#y_test = keras.utils.to_categorical(y_test, num_classes)

def lr_schedule(epoch):
    """Learning Rate Schedule

    Learning rate is scheduled to be reduced after 80, 120, 160, 180 epochs.
    Called automatically every epoch as part of callbacks during training.

    # Arguments
        epoch (int): The number of epochs

    # Returns
        lr (float32): learning rate
    """
    lr = 1e-3
    if epoch > 180:
        lr *= 0.5e-3
    elif epoch > 160:
        lr *= 1e-3
    elif epoch > 120:
        lr *= 1e-2
    elif epoch > 80:
        lr *= 1e-1
    print('Learning rate: ', lr)
    return lr


model = Sequential()

# 1st Convolutional Layer
model.add(Conv2D(filters=96, input_shape=(224,224,3), kernel_size=(11,11),strides=(4,4), padding='valid'))
model.add(Activation('relu'))
# Pooling
model.add(MaxPooling2D(pool_size=(2,2), strides=(2,2), padding='valid'))
# Batch Normalisation before passing it to the next layer
model.add(BatchNormalization())

# 2nd Convolutional Layer
model.add(Conv2D(filters=256, kernel_size=(11,11), strides=(1,1), padding='valid'))
model.add(Activation('relu'))
# Pooling
model.add(MaxPooling2D(pool_size=(2,2), strides=(2,2), padding='valid'))
# Batch Normalisation
model.add(BatchNormalization())

# 3rd Convolutional Layer
model.add(Conv2D(filters=384, kernel_size=(3,3), strides=(1,1), padding='valid'))
model.add(Activation('relu'))
# Batch Normalisation
model.add(BatchNormalization())

# 4th Convolutional Layer
model.add(Conv2D(filters=384, kernel_size=(3,3), strides=(1,1), padding='valid'))
model.add(Activation('relu'))
# Batch Normalisation
model.add(BatchNormalization())

# 5th Convolutional Layer
model.add(Conv2D(filters=256, kernel_size=(3,3), strides=(1,1), padding='valid'))
model.add(Activation('relu'))
# Pooling
model.add(MaxPooling2D(pool_size=(2,2), strides=(2,2), padding='valid'))
# Batch Normalisation
model.add(BatchNormalization())

# Passing it to a dense layer
model.add(Flatten())
# 1st Dense Layer
model.add(Dense(4096, input_shape=(224*224*3,)))
model.add(Activation('relu'))
# Add Dropout to prevent overfitting
model.add(Dropout(0.4))
# Batch Normalisation
model.add(BatchNormalization())

# 2nd Dense Layer
model.add(Dense(4096))
model.add(Activation('relu'))
# Add Dropout
model.add(Dropout(0.4))
# Batch Normalisation
model.add(BatchNormalization())

# 3rd Dense Layer
model.add(Dense(1000))
model.add(Activation('relu'))
# Add Dropout
model.add(Dropout(0.4))
# Batch Normalisation
model.add(BatchNormalization())

# 4th Dense Layer
model.add(Dense(100))
model.add(Activation('relu'))
# Add Dropout
model.add(Dropout(0.4))
# Batch Normalisation
model.add(BatchNormalization())


# Output Layer
model.add(Dense(num_classes))
model.add(Activation('softmax'))

model.summary()
model.compile(loss='categorical_crossentropy', optimizer='adam',metrics=['accuracy'])

# Prepare model model saving directory.
save_dir = '/Users/sayantansikdar/Developer/FRUITS_MASTER/Alexnet_Model'
model_name = 'Alexnet_model_fNet_batch10_epoch150.h5'
saved_model= save_dir + '/' + model_name

# saved_model= save_dir + '/' + model_name
# saved_model_final = load_model(saved_model)
if not os.path.isdir(save_dir):
    os.makedirs(save_dir)
filepath = os.path.join(save_dir, model_name)

# Prepare callbacks for model saving and for learning rate adjustment.
checkpoint = ModelCheckpoint(filepath=filepath,
                             monitor='val_acc',
                             verbose=1,
                             save_best_only=True)

lr_scheduler = LearningRateScheduler(lr_schedule)

lr_reducer = ReduceLROnPlateau(factor=np.sqrt(0.1),
                               cooldown=0,
                               patience=5,
                               min_lr=0.5e-6)

callbacks = [checkpoint, lr_reducer, lr_scheduler]

es = EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=200) #EPOCH CHANGING HERE
mc = ModelCheckpoint(saved_model , monitor='val_accuracy', mode='max', verbose=1, save_best_only=True)


# print(type(trainImages))
# print(type(trainLabels))
# trainImages= np.asarray(trainImages).astype(np.float32)
# trainImages= np.asarray(trainImages).astype(np.float32)
# trainLabels= np.asarray(trainLabels).astype(np.float32)
model.fit(trainImages, trainLabels, batch_size= batch_size, epochs = epochs, verbose=1, validation_split=0.1846, shuffle=True, callbacks=[es, mc])

saved_model_final = load_model(saved_model)


# Score trained model.
preds = saved_model_final.predict(testImages)
print(preds)

end1= time.time()
time_taken= end1- start1

workbook = xlsxwriter.Workbook(result_excel_link)
worksheet = workbook.add_worksheet()

for i in range(0,test_img_num):
        #wb1 = load_workbook('D:/research phd/agriculture research/2nd phase of research/leaf phenotyping/label/results.xlsx')
        #ws1 = wb1.active
        #a = ws1.cell(row= i+1, column=1)
        #a.value = testLabels[i]
        #b = ws1.cell(row= i+1, column=2)
        #b.value = np.argmax(preds [i])
        #ws1.save('D:/research phd/agriculture research/2nd phase of research/leaf phenotyping/label/results.xlsx')
        name = test_img_name[i]
        worksheet.write(i+1, 0 , name)
        a = testLabels[i]
        worksheet.write(i+1, 1 , a)
        b = np.argmax(preds[i])
        worksheet.write(i+1, 2 , b)
        if a==b:
             worksheet.write(i+1, 3 , 1)
        else:
             worksheet.write(i+1, 3 , 0)

# worksheet.write(1, 4 , time_taken)

workbook.close()


11
[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1.]
Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_15 (Conv2D)          (None, 54, 54, 96)        34944     
                                                                 
 activation_30 (Activation)  (None, 54, 54, 96)        0         
                                                                 
 max_pooling2d_9 (MaxPoolin  (None, 27, 27, 96)        0         
 g2D)                                                            
                                                                 
 batch_normalization_27 (Ba  (None, 27, 27, 96)        384       
 tchNormalization)                                               
                                                                 
 conv2d_16 (Conv2D)          (None, 17, 17, 256)       2973952   
                                                                 
 activation_3

Epoch 200

In [5]:
# -*- coding: utf-8 -*-
"""Alexnet_code.ipynb

Automatically generated by Colaboratory.

Original file is located at
    https://colab.research.google.com/drive/1ST_yrafE2p_4JHcbvSpBBYMW1lWIw5hD
"""

from __future__ import print_function
import keras
import pandas as pd
from keras.layers import Dense, Conv2D, BatchNormalization, Activation
from keras.layers import AveragePooling2D, Input, Flatten
from keras.optimizers import Adam
from keras.callbacks import ModelCheckpoint, LearningRateScheduler
from keras.callbacks import ReduceLROnPlateau
from keras.callbacks import EarlyStopping
from keras.callbacks import ModelCheckpoint
from keras.preprocessing.image import ImageDataGenerator
from keras.regularizers import l2
from keras import backend as K
from keras.models import Model
from keras.models import load_model
from keras.utils import to_categorical
import numpy as np
import os
import time
from openpyxl import load_workbook
from keras.models import Sequential
from keras.layers import Dense, Activation, Dropout, Flatten,Conv2D, MaxPooling2D
from keras.layers import BatchNormalization
from PIL import Image
import tensorflow as tf

import xlsxwriter
import cv2
from random import shuffle




start1= time.time()

# Training parameters
batch_size = 10  # orig paper trained all networks with batch_size=128
epochs = 200
data_augmentation = False
num_classes = 12
test_img_num = 2083
result_excel_link = '/Users/sayantansikdar/Developer/FRUITS_MASTER/Alexnet_Result/fruit_batch10_epoch200.xlsx'
Train_DIR= '/Users/sayantansikdar/Developer/FRUITS_MASTER/FINAL DATA_Fruit/TRAIN'
TEST_DIR= '/Users/sayantansikdar/Developer/FRUITS_MASTER/FINAL DATA_Fruit/TEST'
label_link= '/Users/sayantansikdar/Developer/FRUITS_MASTER/FINAL_encoded_file.xlsx'

# Subtracting pixel mean improves accuracy
subtract_pixel_mean = True

# Model parameter
# ----------------------------------------------------------------------------
#           |      | 200-epoch | Orig Paper| 200-epoch | Orig Paper| sec/epoch
# Model     |  n   | ResNet v1 | ResNet v1 | ResNet v2 | ResNet v2 | GTX1080Ti
#           |v1(v2)| %Accuracy | %Accuracy | %Accuracy | %Accuracy | v1 (v2)
# ----------------------------------------------------------------------------
# ResNet20  | 3 (2)| 92.16     | 91.25     | -----     | -----     | 35 (---)
# ResNet32  | 5(NA)| 92.46     | 92.49     | NA        | NA        | 50 ( NA)
# ResNet44  | 7(NA)| 92.50     | 92.83     | NA        | NA        | 70 ( NA)
# ResNet56  | 9 (6)| 92.71     | 93.03     | 93.01     | NA        | 90 (100)
# ResNet110 |18(12)| 92.65     | 93.39+-.16| 93.15     | 93.63     | 165(180)
# ResNet164 |27(18)| -----     | 94.07     | -----     | 94.54     | ---(---)
# ResNet1001| (111)| -----     | 92.39     | -----     | 95.08+-.14| ---(---)
# ---------------------------------------------------------------------------


# Model version
# Orig paper: version = 1 (ResNet v1), Improved ResNet: version = 2 (ResNet v2)
version = 2
n = 2
# Computed depth from supplied model parameter n
if version == 1:
    depth = n * 6 + 2
elif version == 2:
    depth = n * 9 + 2

# Model name, depth and version
model_type = 'AlexNet%dv%d' % (depth, version)

# Load the CIFAR10 data.
##(x_train, y_train), (x_test, y_test) = cifar10.load_data()



def label_img(name):
    # print(name)
    filename1 = os.fsdecode(name)
    # print(filename1)
    s=[]
    s= filename1.split('_')
    # print(s)
    # print(int(s[0]))
    #word_label = name.split('-')[0]
    #if word_label == 'golden_retriever': return np.array([1, 0])
    #elif word_label == 'shetland_sheepdog' : return np.array([0, 1])
    wb = load_workbook(label_link)
    ws = wb.active
    #for cell in ws.columns[1]:  #here column 9 is value is what i am testing which is Column X of my example.
    for x in range(2, 18454):
        #for y in range(1,):
        cell= ws.cell(row=x, column=1)
        # print(cell.value)
        # print(cell.value)
        if cell.value == int(s[0]) :
               #print ws.cell(column=12).value
                #a= ws.columns(row).value
                #b= ws.columns(col).value
                cell2= ws.cell(row=x, column=2)

                label1= cell2.value
                # print(label1)
                break
        #else:
                #print('hi')
    return label1

def load_training_data():
    train_data = []
    for img in os.listdir(Train_DIR):
        #print(img)
        temp2 = img.replace('.png','')
        label = label_img(temp2)
        #print(label)
        path = os.path.join(Train_DIR, img)
        #if "DS_Store" not in path:
        #img = Image.open(path)
        # print(path)
        # print(label)
        img2= cv2.imread(path)
        # cv2_imshow(img2)
        #img = img.convert('L')
        # img = img.resize((IMG_SIZE, IMG_SIZE, 3), Image.ANTIALIAS)
        #path3= 'D:/research phd/agriculture research/2nd phase of research/leaf phenotyping/training_441_segmented_polar_A1_A2_A4'
        resized_img1 = cv2.resize(img2, (224,224), interpolation = cv2.INTER_AREA)
        #cv2.imwrite(path3+ '/' + temp2 , resized_img1 )
        train_data.append([np.array(resized_img1), label])

    shuffle(train_data)
    return train_data


train_data = load_training_data()

# s = [i[1] for i in train_data]
# print(train_data)

# df2 = pd.DataFrame(np.array(df2),columns=['Label'])
# df2['Label'] = pd.Categorical(df2.Label)
IMG_SIZE = 224
# trainImages = np.array([i[0] for i in train_data],dtype=object) #.reshape(-1, IMG_SIZE, IMG_SIZE, 1)
trainImages = np.array([i[0] for i in train_data])
# df = pd.read_csv()
# trainLabels = tf.convert_to_tensor(np.array([i[1] for i in train_data]).astype(np.int_))
trainLabels = np.array([i[1] for i in train_data])
print(trainLabels[2]) #successful till here
#print(trainLabels)
# trainLabels = to_categorical(trainLabels)                                                          #problem here
trainLabels = keras.utils.to_categorical(trainLabels, num_classes)
print(trainLabels[2])



def load_test_data():
    test_data = []
    for img in os.listdir(TEST_DIR):
        temp1 = img.replace('.png','')
        # print(temp1)
        label = label_img(temp1)
        path = os.path.join(TEST_DIR, img)
        # temp1 = img
        #if "DS_Store" not in path:
        #img = Image.open(path)
        img1= cv2.imread(path)
        #img = img.convert('L')
        #img = img.resize((IMG_SIZE, IMG_SIZE, 3), Image.ANTIALIAS)
        resized_img2 = cv2.resize(img1, (224,224), interpolation = cv2.INTER_AREA)
        #path2= 'D:/research phd/agriculture research/2nd phase of research/leaf phenotyping/testing folder_segmented_resized'
        #cv2.imwrite(path2+ '/' + temp1 , resized_img2 )
        test_data.append([np.array(resized_img2), label,temp1])
        # print(label)

    shuffle(test_data)
    return test_data


test_data = load_test_data()
#plt.imshow(test_data[10][0], cmap = 'gist_gray')


# # convert it to tensorflow              ########################
# tensor1 = tf.convert_to_tensor(numpy_array)
# print(tensor1)

# testImages = np.array([i[0] for i in test_data],dtype=object) #.reshape(-1, IMG_SIZE, IMG_SIZE, 1)
# testLabels = tf.convert_to_tensor(np.array([i[1] for i in test_data]).astype(np.int_))

testImages = np.array([i[0] for i in test_data])
testLabels = np.array([i[1] for i in test_data])
test_img_name = np.array([i[2] for i in test_data])


#testLabels = keras.utils.to_categorical(testLabels, num_classes)

# Input image dimensions.
input_shape = trainImages.shape[1:]

# Normalize data.
##trainImages = trainImages.astype('float32') / 255
##testImages = testImages.astype('float32') / 255

# If subtract pixel mean is enabled
##if subtract_pixel_mean:
    ##trainImages_mean = np.mean(trainImages, axis=0)
    ##trainImages -= trainImages_mean
    ##testImages -= trainImages_mean

#print('x_train shape:', x_train.shape)
#print(x_train.shape[0], 'train samples')
#print(x_test.shape[0], 'test samples')
#print('y_train shape:', y_train.shape)

# Convert class vectors to binary class matrices.
#y_train = keras.utils.to_categorical(y_train, num_classes)
#y_test = keras.utils.to_categorical(y_test, num_classes)

def lr_schedule(epoch):
    """Learning Rate Schedule

    Learning rate is scheduled to be reduced after 80, 120, 160, 180 epochs.
    Called automatically every epoch as part of callbacks during training.

    # Arguments
        epoch (int): The number of epochs

    # Returns
        lr (float32): learning rate
    """
    lr = 1e-3
    if epoch > 180:
        lr *= 0.5e-3
    elif epoch > 160:
        lr *= 1e-3
    elif epoch > 120:
        lr *= 1e-2
    elif epoch > 80:
        lr *= 1e-1
    print('Learning rate: ', lr)
    return lr


model = Sequential()

# 1st Convolutional Layer
model.add(Conv2D(filters=96, input_shape=(224,224,3), kernel_size=(11,11),strides=(4,4), padding='valid'))
model.add(Activation('relu'))
# Pooling
model.add(MaxPooling2D(pool_size=(2,2), strides=(2,2), padding='valid'))
# Batch Normalisation before passing it to the next layer
model.add(BatchNormalization())

# 2nd Convolutional Layer
model.add(Conv2D(filters=256, kernel_size=(11,11), strides=(1,1), padding='valid'))
model.add(Activation('relu'))
# Pooling
model.add(MaxPooling2D(pool_size=(2,2), strides=(2,2), padding='valid'))
# Batch Normalisation
model.add(BatchNormalization())

# 3rd Convolutional Layer
model.add(Conv2D(filters=384, kernel_size=(3,3), strides=(1,1), padding='valid'))
model.add(Activation('relu'))
# Batch Normalisation
model.add(BatchNormalization())

# 4th Convolutional Layer
model.add(Conv2D(filters=384, kernel_size=(3,3), strides=(1,1), padding='valid'))
model.add(Activation('relu'))
# Batch Normalisation
model.add(BatchNormalization())

# 5th Convolutional Layer
model.add(Conv2D(filters=256, kernel_size=(3,3), strides=(1,1), padding='valid'))
model.add(Activation('relu'))
# Pooling
model.add(MaxPooling2D(pool_size=(2,2), strides=(2,2), padding='valid'))
# Batch Normalisation
model.add(BatchNormalization())

# Passing it to a dense layer
model.add(Flatten())
# 1st Dense Layer
model.add(Dense(4096, input_shape=(224*224*3,)))
model.add(Activation('relu'))
# Add Dropout to prevent overfitting
model.add(Dropout(0.4))
# Batch Normalisation
model.add(BatchNormalization())

# 2nd Dense Layer
model.add(Dense(4096))
model.add(Activation('relu'))
# Add Dropout
model.add(Dropout(0.4))
# Batch Normalisation
model.add(BatchNormalization())

# 3rd Dense Layer
model.add(Dense(1000))
model.add(Activation('relu'))
# Add Dropout
model.add(Dropout(0.4))
# Batch Normalisation
model.add(BatchNormalization())

# 4th Dense Layer
model.add(Dense(100))
model.add(Activation('relu'))
# Add Dropout
model.add(Dropout(0.4))
# Batch Normalisation
model.add(BatchNormalization())


# Output Layer
model.add(Dense(num_classes))
model.add(Activation('softmax'))

model.summary()
model.compile(loss='categorical_crossentropy', optimizer='adam',metrics=['accuracy'])

# Prepare model model saving directory.
save_dir = '/Users/sayantansikdar/Developer/FRUITS_MASTER/Alexnet_Model'
model_name = 'Alexnet_model_fNet_batch10_epoch200.h5'
saved_model= save_dir + '/' + model_name

# saved_model= save_dir + '/' + model_name
# saved_model_final = load_model(saved_model)
if not os.path.isdir(save_dir):
    os.makedirs(save_dir)
filepath = os.path.join(save_dir, model_name)

# Prepare callbacks for model saving and for learning rate adjustment.
checkpoint = ModelCheckpoint(filepath=filepath,
                             monitor='val_acc',
                             verbose=1,
                             save_best_only=True)

lr_scheduler = LearningRateScheduler(lr_schedule)

lr_reducer = ReduceLROnPlateau(factor=np.sqrt(0.1),
                               cooldown=0,
                               patience=5,
                               min_lr=0.5e-6)

callbacks = [checkpoint, lr_reducer, lr_scheduler]

es = EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=200) #EPOCH CHANGING HERE
mc = ModelCheckpoint(saved_model , monitor='val_accuracy', mode='max', verbose=1, save_best_only=True)


# print(type(trainImages))
# print(type(trainLabels))
# trainImages= np.asarray(trainImages).astype(np.float32)
# trainImages= np.asarray(trainImages).astype(np.float32)
# trainLabels= np.asarray(trainLabels).astype(np.float32)
model.fit(trainImages, trainLabels, batch_size= batch_size, epochs = epochs, verbose=1, validation_split=0.1846, shuffle=True, callbacks=[es, mc])

saved_model_final = load_model(saved_model)


# Score trained model.
preds = saved_model_final.predict(testImages)
print(preds)

end1= time.time()
time_taken= end1- start1

workbook = xlsxwriter.Workbook(result_excel_link)
worksheet = workbook.add_worksheet()

for i in range(0,test_img_num):
        #wb1 = load_workbook('D:/research phd/agriculture research/2nd phase of research/leaf phenotyping/label/results.xlsx')
        #ws1 = wb1.active
        #a = ws1.cell(row= i+1, column=1)
        #a.value = testLabels[i]
        #b = ws1.cell(row= i+1, column=2)
        #b.value = np.argmax(preds [i])
        #ws1.save('D:/research phd/agriculture research/2nd phase of research/leaf phenotyping/label/results.xlsx')
        name = test_img_name[i]
        worksheet.write(i+1, 0 , name)
        a = testLabels[i]
        worksheet.write(i+1, 1 , a)
        b = np.argmax(preds[i])
        worksheet.write(i+1, 2 , b)
        if a==b:
             worksheet.write(i+1, 3 , 1)
        else:
             worksheet.write(i+1, 3 , 0)

# worksheet.write(1, 4 , time_taken)

workbook.close()

4
[0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0.]
Model: "sequential_4"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_20 (Conv2D)          (None, 54, 54, 96)        34944     
                                                                 
 activation_40 (Activation)  (None, 54, 54, 96)        0         
                                                                 
 max_pooling2d_12 (MaxPooli  (None, 27, 27, 96)        0         
 ng2D)                                                           
                                                                 
 batch_normalization_36 (Ba  (None, 27, 27, 96)        384       
 tchNormalization)                                               
                                                                 
 conv2d_21 (Conv2D)          (None, 17, 17, 256)       2973952   
                                                                 
 activation_41

In [6]:
# -*- coding: utf-8 -*-
"""Alexnet_code.ipynb

Automatically generated by Colaboratory.

Original file is located at
    https://colab.research.google.com/drive/1ST_yrafE2p_4JHcbvSpBBYMW1lWIw5hD
"""

from __future__ import print_function
import keras
import pandas as pd
from keras.layers import Dense, Conv2D, BatchNormalization, Activation
from keras.layers import AveragePooling2D, Input, Flatten
from keras.optimizers import Adam
from keras.callbacks import ModelCheckpoint, LearningRateScheduler
from keras.callbacks import ReduceLROnPlateau
from keras.callbacks import EarlyStopping
from keras.callbacks import ModelCheckpoint
from keras.preprocessing.image import ImageDataGenerator
from keras.regularizers import l2
from keras import backend as K
from keras.models import Model
from keras.models import load_model
from keras.utils import to_categorical
import numpy as np
import os
import time
from openpyxl import load_workbook
from keras.models import Sequential
from keras.layers import Dense, Activation, Dropout, Flatten,Conv2D, MaxPooling2D
from keras.layers import BatchNormalization
from PIL import Image
import tensorflow as tf

import xlsxwriter
import cv2
from random import shuffle




start1= time.time()

# Training parameters
batch_size = 10  # orig paper trained all networks with batch_size=128
epochs = 250
data_augmentation = False
num_classes = 12
test_img_num = 2083
result_excel_link = '/Users/sayantansikdar/Developer/FRUITS_MASTER/Alexnet_Result/fruit_batch10_epoch250.xlsx'
Train_DIR= '/Users/sayantansikdar/Developer/FRUITS_MASTER/FINAL DATA_Fruit/TRAIN'
TEST_DIR= '/Users/sayantansikdar/Developer/FRUITS_MASTER/FINAL DATA_Fruit/TEST'
label_link= '/Users/sayantansikdar/Developer/FRUITS_MASTER/FINAL_encoded_file.xlsx'

# Subtracting pixel mean improves accuracy
subtract_pixel_mean = True

# Model parameter
# ----------------------------------------------------------------------------
#           |      | 200-epoch | Orig Paper| 200-epoch | Orig Paper| sec/epoch
# Model     |  n   | ResNet v1 | ResNet v1 | ResNet v2 | ResNet v2 | GTX1080Ti
#           |v1(v2)| %Accuracy | %Accuracy | %Accuracy | %Accuracy | v1 (v2)
# ----------------------------------------------------------------------------
# ResNet20  | 3 (2)| 92.16     | 91.25     | -----     | -----     | 35 (---)
# ResNet32  | 5(NA)| 92.46     | 92.49     | NA        | NA        | 50 ( NA)
# ResNet44  | 7(NA)| 92.50     | 92.83     | NA        | NA        | 70 ( NA)
# ResNet56  | 9 (6)| 92.71     | 93.03     | 93.01     | NA        | 90 (100)
# ResNet110 |18(12)| 92.65     | 93.39+-.16| 93.15     | 93.63     | 165(180)
# ResNet164 |27(18)| -----     | 94.07     | -----     | 94.54     | ---(---)
# ResNet1001| (111)| -----     | 92.39     | -----     | 95.08+-.14| ---(---)
# ---------------------------------------------------------------------------


# Model version
# Orig paper: version = 1 (ResNet v1), Improved ResNet: version = 2 (ResNet v2)
version = 2
n = 2
# Computed depth from supplied model parameter n
if version == 1:
    depth = n * 6 + 2
elif version == 2:
    depth = n * 9 + 2

# Model name, depth and version
model_type = 'AlexNet%dv%d' % (depth, version)

# Load the CIFAR10 data.
##(x_train, y_train), (x_test, y_test) = cifar10.load_data()



def label_img(name):
    # print(name)
    filename1 = os.fsdecode(name)
    # print(filename1)
    s=[]
    s= filename1.split('_')
    # print(s)
    # print(int(s[0]))
    #word_label = name.split('-')[0]
    #if word_label == 'golden_retriever': return np.array([1, 0])
    #elif word_label == 'shetland_sheepdog' : return np.array([0, 1])
    wb = load_workbook(label_link)
    ws = wb.active
    #for cell in ws.columns[1]:  #here column 9 is value is what i am testing which is Column X of my example.
    for x in range(2, 18454):
        #for y in range(1,):
        cell= ws.cell(row=x, column=1)
        # print(cell.value)
        # print(cell.value)
        if cell.value == int(s[0]) :
               #print ws.cell(column=12).value
                #a= ws.columns(row).value
                #b= ws.columns(col).value
                cell2= ws.cell(row=x, column=2)

                label1= cell2.value
                # print(label1)
                break
        #else:
                #print('hi')
    return label1

def load_training_data():
    train_data = []
    for img in os.listdir(Train_DIR):
        #print(img)
        temp2 = img.replace('.png','')
        label = label_img(temp2)
        #print(label)
        path = os.path.join(Train_DIR, img)
        #if "DS_Store" not in path:
        #img = Image.open(path)
        # print(path)
        # print(label)
        img2= cv2.imread(path)
        # cv2_imshow(img2)
        #img = img.convert('L')
        # img = img.resize((IMG_SIZE, IMG_SIZE, 3), Image.ANTIALIAS)
        #path3= 'D:/research phd/agriculture research/2nd phase of research/leaf phenotyping/training_441_segmented_polar_A1_A2_A4'
        resized_img1 = cv2.resize(img2, (224,224), interpolation = cv2.INTER_AREA)
        #cv2.imwrite(path3+ '/' + temp2 , resized_img1 )
        train_data.append([np.array(resized_img1), label])

    shuffle(train_data)
    return train_data


train_data = load_training_data()

# s = [i[1] for i in train_data]
# print(train_data)

# df2 = pd.DataFrame(np.array(df2),columns=['Label'])
# df2['Label'] = pd.Categorical(df2.Label)
IMG_SIZE = 224
# trainImages = np.array([i[0] for i in train_data],dtype=object) #.reshape(-1, IMG_SIZE, IMG_SIZE, 1)
trainImages = np.array([i[0] for i in train_data])
# df = pd.read_csv()
# trainLabels = tf.convert_to_tensor(np.array([i[1] for i in train_data]).astype(np.int_))
trainLabels = np.array([i[1] for i in train_data])
print(trainLabels[2]) #successful till here
#print(trainLabels)
# trainLabels = to_categorical(trainLabels)                                                          #problem here
trainLabels = keras.utils.to_categorical(trainLabels, num_classes)
print(trainLabels[2])



def load_test_data():
    test_data = []
    for img in os.listdir(TEST_DIR):
        temp1 = img.replace('.png','')
        # print(temp1)
        label = label_img(temp1)
        path = os.path.join(TEST_DIR, img)
        # temp1 = img
        #if "DS_Store" not in path:
        #img = Image.open(path)
        img1= cv2.imread(path)
        #img = img.convert('L')
        #img = img.resize((IMG_SIZE, IMG_SIZE, 3), Image.ANTIALIAS)
        resized_img2 = cv2.resize(img1, (224,224), interpolation = cv2.INTER_AREA)
        #path2= 'D:/research phd/agriculture research/2nd phase of research/leaf phenotyping/testing folder_segmented_resized'
        #cv2.imwrite(path2+ '/' + temp1 , resized_img2 )
        test_data.append([np.array(resized_img2), label,temp1])
        # print(label)

    shuffle(test_data)
    return test_data


test_data = load_test_data()
#plt.imshow(test_data[10][0], cmap = 'gist_gray')


# # convert it to tensorflow              ########################
# tensor1 = tf.convert_to_tensor(numpy_array)
# print(tensor1)

# testImages = np.array([i[0] for i in test_data],dtype=object) #.reshape(-1, IMG_SIZE, IMG_SIZE, 1)
# testLabels = tf.convert_to_tensor(np.array([i[1] for i in test_data]).astype(np.int_))

testImages = np.array([i[0] for i in test_data])
testLabels = np.array([i[1] for i in test_data])
test_img_name = np.array([i[2] for i in test_data])


#testLabels = keras.utils.to_categorical(testLabels, num_classes)

# Input image dimensions.
input_shape = trainImages.shape[1:]

# Normalize data.
##trainImages = trainImages.astype('float32') / 255
##testImages = testImages.astype('float32') / 255

# If subtract pixel mean is enabled
##if subtract_pixel_mean:
    ##trainImages_mean = np.mean(trainImages, axis=0)
    ##trainImages -= trainImages_mean
    ##testImages -= trainImages_mean

#print('x_train shape:', x_train.shape)
#print(x_train.shape[0], 'train samples')
#print(x_test.shape[0], 'test samples')
#print('y_train shape:', y_train.shape)

# Convert class vectors to binary class matrices.
#y_train = keras.utils.to_categorical(y_train, num_classes)
#y_test = keras.utils.to_categorical(y_test, num_classes)

def lr_schedule(epoch):
    """Learning Rate Schedule

    Learning rate is scheduled to be reduced after 80, 120, 160, 180 epochs.
    Called automatically every epoch as part of callbacks during training.

    # Arguments
        epoch (int): The number of epochs

    # Returns
        lr (float32): learning rate
    """
    lr = 1e-3
    if epoch > 180:
        lr *= 0.5e-3
    elif epoch > 160:
        lr *= 1e-3
    elif epoch > 120:
        lr *= 1e-2
    elif epoch > 80:
        lr *= 1e-1
    print('Learning rate: ', lr)
    return lr


model = Sequential()

# 1st Convolutional Layer
model.add(Conv2D(filters=96, input_shape=(224,224,3), kernel_size=(11,11),strides=(4,4), padding='valid'))
model.add(Activation('relu'))
# Pooling
model.add(MaxPooling2D(pool_size=(2,2), strides=(2,2), padding='valid'))
# Batch Normalisation before passing it to the next layer
model.add(BatchNormalization())

# 2nd Convolutional Layer
model.add(Conv2D(filters=256, kernel_size=(11,11), strides=(1,1), padding='valid'))
model.add(Activation('relu'))
# Pooling
model.add(MaxPooling2D(pool_size=(2,2), strides=(2,2), padding='valid'))
# Batch Normalisation
model.add(BatchNormalization())

# 3rd Convolutional Layer
model.add(Conv2D(filters=384, kernel_size=(3,3), strides=(1,1), padding='valid'))
model.add(Activation('relu'))
# Batch Normalisation
model.add(BatchNormalization())

# 4th Convolutional Layer
model.add(Conv2D(filters=384, kernel_size=(3,3), strides=(1,1), padding='valid'))
model.add(Activation('relu'))
# Batch Normalisation
model.add(BatchNormalization())

# 5th Convolutional Layer
model.add(Conv2D(filters=256, kernel_size=(3,3), strides=(1,1), padding='valid'))
model.add(Activation('relu'))
# Pooling
model.add(MaxPooling2D(pool_size=(2,2), strides=(2,2), padding='valid'))
# Batch Normalisation
model.add(BatchNormalization())

# Passing it to a dense layer
model.add(Flatten())
# 1st Dense Layer
model.add(Dense(4096, input_shape=(224*224*3,)))
model.add(Activation('relu'))
# Add Dropout to prevent overfitting
model.add(Dropout(0.4))
# Batch Normalisation
model.add(BatchNormalization())

# 2nd Dense Layer
model.add(Dense(4096))
model.add(Activation('relu'))
# Add Dropout
model.add(Dropout(0.4))
# Batch Normalisation
model.add(BatchNormalization())

# 3rd Dense Layer
model.add(Dense(1000))
model.add(Activation('relu'))
# Add Dropout
model.add(Dropout(0.4))
# Batch Normalisation
model.add(BatchNormalization())

# 4th Dense Layer
model.add(Dense(100))
model.add(Activation('relu'))
# Add Dropout
model.add(Dropout(0.4))
# Batch Normalisation
model.add(BatchNormalization())


# Output Layer
model.add(Dense(num_classes))
model.add(Activation('softmax'))

model.summary()
model.compile(loss='categorical_crossentropy', optimizer='adam',metrics=['accuracy'])

# Prepare model model saving directory.
save_dir = '/Users/sayantansikdar/Developer/FRUITS_MASTER/Alexnet_Model'
model_name = 'Alexnet_model_fNet_batch10_epoch250.h5'
saved_model= save_dir + '/' + model_name

# saved_model= save_dir + '/' + model_name
# saved_model_final = load_model(saved_model)
if not os.path.isdir(save_dir):
    os.makedirs(save_dir)
filepath = os.path.join(save_dir, model_name)

# Prepare callbacks for model saving and for learning rate adjustment.
checkpoint = ModelCheckpoint(filepath=filepath,
                             monitor='val_acc',
                             verbose=1,
                             save_best_only=True)

lr_scheduler = LearningRateScheduler(lr_schedule)

lr_reducer = ReduceLROnPlateau(factor=np.sqrt(0.1),
                               cooldown=0,
                               patience=5,
                               min_lr=0.5e-6)

callbacks = [checkpoint, lr_reducer, lr_scheduler]

es = EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=200) #EPOCH CHANGING HERE
mc = ModelCheckpoint(saved_model , monitor='val_accuracy', mode='max', verbose=1, save_best_only=True)


# print(type(trainImages))
# print(type(trainLabels))
# trainImages= np.asarray(trainImages).astype(np.float32)
# trainImages= np.asarray(trainImages).astype(np.float32)
# trainLabels= np.asarray(trainLabels).astype(np.float32)
model.fit(trainImages, trainLabels, batch_size= batch_size, epochs = epochs, verbose=1, validation_split=0.1846, shuffle=True, callbacks=[es, mc])

saved_model_final = load_model(saved_model)


# Score trained model.
preds = saved_model_final.predict(testImages)
print(preds)

end1= time.time()
time_taken= end1- start1

workbook = xlsxwriter.Workbook(result_excel_link)
worksheet = workbook.add_worksheet()

for i in range(0,test_img_num):
        #wb1 = load_workbook('D:/research phd/agriculture research/2nd phase of research/leaf phenotyping/label/results.xlsx')
        #ws1 = wb1.active
        #a = ws1.cell(row= i+1, column=1)
        #a.value = testLabels[i]
        #b = ws1.cell(row= i+1, column=2)
        #b.value = np.argmax(preds [i])
        #ws1.save('D:/research phd/agriculture research/2nd phase of research/leaf phenotyping/label/results.xlsx')
        name = test_img_name[i]
        worksheet.write(i+1, 0 , name)
        a = testLabels[i]
        worksheet.write(i+1, 1 , a)
        b = np.argmax(preds[i])
        worksheet.write(i+1, 2 , b)
        if a==b:
             worksheet.write(i+1, 3 , 1)
        else:
             worksheet.write(i+1, 3 , 0)

# worksheet.write(1, 4 , time_taken)

workbook.close()

8
[0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0.]
Model: "sequential_5"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_25 (Conv2D)          (None, 54, 54, 96)        34944     
                                                                 
 activation_50 (Activation)  (None, 54, 54, 96)        0         
                                                                 
 max_pooling2d_15 (MaxPooli  (None, 27, 27, 96)        0         
 ng2D)                                                           
                                                                 
 batch_normalization_45 (Ba  (None, 27, 27, 96)        384       
 tchNormalization)                                               
                                                                 
 conv2d_26 (Conv2D)          (None, 17, 17, 256)       2973952   
                                                                 
 activation_51